# NBL-V3 Ball-Only Training
## Specialized model for basketball detection

**Why separate ball model?**
- Basketball is a SMALL object (harder to detect)
- Needs ultra-low confidence threshold (0.05)
- Benefits from higher resolution (1280px)
- Can optimize specifically for small object detection

# Install Libraries

In [ ]:
!pip install ultralytics
!pip install roboflow

In [ ]:
!pip freeze | grep roboflow

# Imports

In [ ]:
from roboflow import Roboflow
import shutil
import yaml

# Get Dataset

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ZzD21wz5oTPdE0fhb04C")
project = rf.workspace("tomatoes-iicln").project("nbl")
version = project.version(3)
dataset = version.download("yolov11")

In [ ]:
# 🔧 FIX: Correct data.yaml paths for Google Colab
import yaml
import os

yaml_path = '/content/NBL-3/data_ball_only.yaml' if os.path.exists('/content/NBL-3/data_ball_only.yaml') else '/content/NBL-3/data_players_only.yaml' if os.path.exists('/content/NBL-3/data_players_only.yaml') else '/content/NBL-3/data.yaml'
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# Set absolute paths for Colab
data['train'] = '/content/NBL-3/train/images'
data['val'] = '/content/NBL-3/valid/images'
data['test'] = '/content/NBL-3/test/images'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print(f"✅ Updated {yaml_path} with absolute paths:")
print(f"Train: {data['train']}")
print(f"Val: {data['val']}")


In [ ]:
shutil.move("NBL-3/train",
"NBL-3/NBL-3/train",
)
shutil.move("NBL-3/valid",
"NBL-3/NBL-3/valid",
)

# Filter Dataset for Ball Only

We need to modify data.yaml to only include 'basketball' class

In [ ]:
# Read original data.yaml
with open('/content/NBL-3/data.yaml', 'r') as f:
    data = yaml.safe_load(f)

print("Original classes:", data['names'])

# Find basketball class index
basketball_idx = data['names'].index('basketball')
print(f"Basketball is class {basketball_idx}")

# Create ball-only data.yaml
ball_data = {
    'train': data['train'],
    'val': data['val'],
    'test': data['test'],
    'nc': 1,
    'names': ['basketball']
}

# Save ball-only config
with open('/content/NBL-3/data_ball_only.yaml', 'w') as f:
    yaml.dump(ball_data, f)

print("\nCreated data_ball_only.yaml")
print("Note: Training will only use images with basketball annotations")

# Training - Optimized for Small Objects

In [ ]:
from ultralytics import YOLO

# Use YOLOv11n - best for small object detection
model = YOLO('yolo11n.pt')

results = model.train(
    data='/content/NBL-3/data_ball_only.yaml',
    epochs=150,  # More epochs for small object
    batch=8,     # Smaller batch for higher resolution
    imgsz=1280,  # HIGHER resolution for small ball
    patience=75,
    device=0,
    workers=8,
    project='nbl_v3_ball_only',
    name='train',
    exist_ok=False,
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=0,
    deterministic=True,
    val=True,
    plots=True,
    # Small object optimization
    mosaic=1.0,      # Mosaic helps with small objects
    scale=0.9,       # More aggressive scaling
    copy_paste=0.3,  # Copy-paste augmentation for small objects
)

# Download Model

Model will be saved as: `nbl_v3_ball_only/train/weights/best.pt`

Use this in your config:
```python
BALL_DETECTOR_PATH = 'models/nbl_v3_ball.pt'
```